# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import warnings
import numpy as np
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,11.07,95,78,2.75,AU,1701198289
1,1,carnarvon,-24.8667,113.6333,21.41,64,100,8.65,AU,1701198289
2,2,egilsstadir,65.2653,-14.3948,4.95,75,75,5.14,IS,1701198290
3,3,torbat-e heydariyeh,35.2740,59.2195,11.08,63,97,2.89,IR,1701198290
4,4,waitangi,-43.9535,-176.5597,16.57,90,74,3.13,NZ,1701198291


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot_1 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo= True,
    tiles='OSM',
    frame_width= 800,
    frame_height= 600,
    size= 'Humidity',
    #hover_cols= ['city, "humidity'],
    color = 'City',
    alpha = .5,
    scale = 1
   
   
)

# Display the map
city_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df['Max Temp'] < 80, :]
ideal_city_df = city_data_df.loc[city_data_df['Max Temp'] > 70, :]
ideal_city_df = city_data_df.loc[city_data_df['Wind Speed'] < 10, :]
ideal_city_df = city_data_df.loc[city_data_df['Cloudiness'] == 0, :]

# Drop any rows with null values
ideal_city_df.dropna(inplace = True)

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,hasaki,35.7333,140.8333,2.22,37,0,3.31,JP,1701198293
15,15,islamkot,24.6997,70.1770,21.28,37,0,3.13,PK,1701198294
26,26,grytviken,-54.2811,-36.5092,1.29,70,0,3.69,GS,1701198297
38,38,ghat,24.9647,10.1728,19.89,15,0,1.19,LY,1701198300
41,41,sur,22.5667,59.5289,24.38,68,0,3.94,OM,1701198301
...,...,...,...,...,...,...,...,...,...,...
502,502,jackson,42.4165,-122.8345,2.93,65,0,0.00,US,1701198468
506,506,kidal,18.4411,1.4078,26.71,12,0,3.87,ML,1701198469
516,516,buxerolles,46.6167,0.4833,6.97,87,0,0.51,FR,1701198471
529,529,kill devil hills,36.0307,-75.6760,8.88,45,0,7.20,US,1701198475


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hasaki,JP,35.7333,140.8333,37,NaN
15,islamkot,PK,24.6997,70.1770,37,NaN
26,grytviken,GS,-54.2811,-36.5092,70,NaN
38,ghat,LY,24.9647,10.1728,15,NaN
41,sur,OM,22.5667,59.5289,68,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    target_lat = row['Lat']
    target_lng = row['Lng']
    
    params = {
    "apiKey": geoapify_key,
    "limit": 1,
    "categories": "accommodation.hotel"
    }
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{target_lng},{target_lat},{radius}"
    params["bias"] = f"proximity:{target_lng},{target_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hasaki - nearest hotel: 大新旅館
islamkot - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
sur - nearest hotel: Sur Hotel
aswan - nearest hotel: Yaseen hotel
plastun - nearest hotel: Россияночка
cabo san lucas - nearest hotel: Comfort Rooms
dauphin - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
bodo - nearest hotel: Zefyr hotell
murzuq - nearest hotel: فندق باريس - مرزق
el bauga - nearest hotel: No hotel found
rice lake - nearest hotel: Paul Bunyan Resort
retreat - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
tazacorte - nearest hotel: App Leyma
mar del plata - nearest hotel: Gran hotel Mónaco
devils lake - nearest hotel: Holiday Inn Express
al wajh - nearest hotel: فندق رالين 2
ad dabbah - nearest hotel: No hotel found
junagarh - nearest hotel: No hotel found
bayside - nearest hotel: Hotel Arcata
griffith - nearest hotel: Quality Inn & Suites Hamm

,City,Country,Lat,Lng,Humidity,Hotel Name
11,hasaki,JP,35.7333,140.8333,37,大新旅館
15,islamkot,PK,24.6997,70.1770,37,No hotel found
26,grytviken,GS,-54.2811,-36.5092,70,No hotel found
38,ghat,LY,24.9647,10.1728,15,فندق تاسيلي
41,sur,OM,22.5667,59.5289,68,Sur Hotel
...,...,...,...,...,...,...
502,jackson,US,42.4165,-122.8345,65,Brookside
506,kidal,ML,18.4411,1.4078,12,No hotel found
516,buxerolles,FR,46.6167,0.4833,87,Château du Clos de la Ribaudière
529,kill devil hills,US,36.0307,-75.6760,45,Mariner Days Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat',
    geo= True,
    tiles='OSM',
    frame_width= 800,
    frame_height= 600,
    size= 'Humidity',
    color = 'City',
    alpha = .5,
    scale = 1,
    hover_cols= ["Hotel Name", "Country"]
)

# Display the map
hotel_df_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)